In [ ]:
bronze_path = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/bronze/"
silver_path = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/silver/"
gold_path   = "abfss://<container>@<storage-account>.dfs.core.windows.net/bikestore/gold/"
resource_path = "/Volumes/<CATALOG>/<SCHEMA>/bikestore_resource/origem"
resource_path_volume = '/Volumes/aulas_databricks/azure/bikestore_resource/origem/'


In [0]:
%python
bronze_map = {
    #"tmp_bronze_brands":      f"{bronze_path}/brands/",
    #"tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    #"tmp_bronze_order_items": f"{bronze_path}/order_items/",
    #"tmp_bronze_orders":      f"{bronze_path}/orders/",
    #"tmp_bronze_products":    f"{bronze_path}/products/",
    #"tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    #"tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    #"tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%python
df_customer_silver = spark.sql("""

SELECT 
  CT.customer_id
  ,CT.first_name
  ,CT.last_name
  ,CT.phone
  ,CT.email
  ,CT.street
  ,CT.city
  ,CT.state
  ,CT.zip_code
FROM  tmp_bronze_customers CT
WHERE  1=1
AND  CT.phone IS NOT NULL -- vazio 
AND  CT.phone NOT IN ('NULL','NULL ')-- texto null
and  CT.email IS NOT NULL -- vazio 
AND  CT.email NOT IN ('NULL','NULL ')
         
                              
                              """)

# salvar em Delta na silver 
df_customer_silver.write\
    .mode('overwrite')\
    .format('delta')\
    .option('mergeSchema','true')\
    .save(f'{silver_path}/customer')
